<a href="https://colab.research.google.com/github/daanbee/tobigs-22nd/blob/main/%ED%88%AC%EB%B9%85%EC%8A%A4_Week2_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [2]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [3]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [4]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [5]:
# 사전 확률 구하는 코드를 작성해주세요.
from collections import Counter

sentiment_counts = Counter(df['sentiment'])

total_samples = len(df)

P_positive = sentiment_counts['positive'] / total_samples
P_negative = sentiment_counts['negative'] / total_samples
P_neutral = sentiment_counts['neutral'] / total_samples

가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [7]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [8]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [9]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [10]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요
word_counts_per_sentiment = frequency_matrix.groupby('sentiment').sum()

total_words_per_sentiment = word_counts_per_sentiment.sum(axis=1)

conditional_probabilities = word_counts_per_sentiment.div(total_words_per_sentiment, axis=0)

conditional_probabilities.head()

,absolutely,acceptable,after,again,average,better,brand,broke,but,buy,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
sentiment,,,,,,,,,,,,,,,,,,,,,
negative,0.000000,0.000,0.041667,0.041667,0.000000,0.000000,0.041667,0.041667,0.000000,0.041667,...,0.083333,0.000000,0.041667,0.041667,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667
neutral,0.000000,0.125,0.000000,0.000000,0.041667,0.083333,0.000000,0.000000,0.125000,0.000000,...,0.041667,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
positive,0.038462,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.038462,0.038462,0.000000,0.076923,0.076923,0.000000


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [15]:
# 가능도 구하는 코드를 작성해주세요.
target_review1_words = ['love', 'great', 'awesome']
target_review2_words = ['poor', 'broke', 'terrible']

# 각 단어가 감정별로 나타난 가능도를 구하는 함수
def calculate_likelihood(words, sentiment):
    likelihoods = []
    for word in words:
        if word in word_counts_per_sentiment.columns:
            word_count = word_counts_per_sentiment[word][sentiment]
            total_count = total_words_per_sentiment[sentiment]
            likelihood = word_count / total_count if total_count > 0 else 0
        else:
            likelihood = 0
        likelihoods.append(likelihood)
    return likelihoods

# 첫 번째 리뷰에 대한 각 감정별 가능도 계산
likelihoods_positive_review1 = calculate_likelihood(target_review1_words, 'positive')
likelihoods_negative_review1 = calculate_likelihood(target_review1_words, 'negative')
likelihoods_neutral_review1 = calculate_likelihood(target_review1_words, 'neutral')

# 두 번째 리뷰에 대한 각 감정별 가능도 계산
likelihoods_positive_review2 = calculate_likelihood(target_review2_words, 'positive')
likelihoods_negative_review2 = calculate_likelihood(target_review2_words, 'negative')
likelihoods_neutral_review2 = calculate_likelihood(target_review2_words, 'neutral')

# 결과 출력
print(f"target_review1\nLikelihoods for 'positive': {likelihoods_positive_review1}")
print(f"Likelihoods for 'negative': {likelihoods_negative_review1}")
print(f"Likelihoods for 'neutral': {likelihoods_neutral_review1}")

print(f"\ntarget_review2\nLikelihoods for 'positive': {likelihoods_positive_review2}")
print(f"Likelihoods for 'negative': {likelihoods_negative_review2}")
print(f"Likelihoods for 'neutral': {likelihoods_neutral_review2}")

target_review1
Likelihoods for 'positive': [0.11538461538461539, 0.15384615384615385, 0]
Likelihoods for 'negative': [0.0, 0.0, 0]
Likelihoods for 'neutral': [0.0, 0.0, 0]

target_review2
Likelihoods for 'positive': [0.0, 0.0, 0.0]
Likelihoods for 'negative': [0.08333333333333333, 0.041666666666666664, 0.08333333333333333]
Likelihoods for 'neutral': [0.0, 0.0, 0.041666666666666664]


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [16]:
import numpy as np
# 최종 확률 구하는 코드를 작성해주세요.
P_review1_given_positive = np.prod(likelihoods_positive_review1)
P_review1_given_negative = np.prod(likelihoods_negative_review1)
P_review1_given_neutral = np.prod(likelihoods_neutral_review1)

P_review2_given_positive = np.prod(likelihoods_positive_review2)
P_review2_given_negative = np.prod(likelihoods_negative_review2)
P_review2_given_neutral = np.prod(likelihoods_neutral_review2)

P_target_review1 = (P_review1_given_positive * P_positive +
                    P_review1_given_negative * P_negative +
                    P_review1_given_neutral * P_neutral)

P_target_review2 = (P_review2_given_positive * P_positive +
                    P_review2_given_negative * P_negative +
                    P_review2_given_neutral * P_neutral)

P_positive_given_review1 = (P_review1_given_positive * P_positive) / P_target_review1 if P_target_review1 != 0 else 0
P_negative_given_review1 = (P_review1_given_negative * P_negative) / P_target_review1 if P_target_review1 != 0 else 0
P_neutral_given_review1 = (P_review1_given_neutral * P_neutral) / P_target_review1 if P_target_review1 != 0 else 0

P_positive_given_review2 = (P_review2_given_positive * P_positive) / P_target_review2 if P_target_review2 != 0 else 0
P_negative_given_review2 = (P_review2_given_negative * P_negative) / P_target_review2 if P_target_review2 != 0 else 0
P_neutral_given_review2 = (P_review2_given_neutral * P_neutral) / P_target_review2 if P_target_review2 != 0 else 0

print(f"target_review1\nP(positive|target_review1): {P_positive_given_review1:.4f}")
print(f"P(negative|target_review1): {P_negative_given_review1:.4f}")
print(f"P(neutral|target_review1): {P_neutral_given_review1:.4f}")

print(f"\ntarget_review2\nP(positive|target_review2): {P_positive_given_review2:.4f}")
print(f"P(negative|target_review2): {P_negative_given_review2:.4f}")
print(f"P(neutral|target_review2): {P_neutral_given_review2:.4f}")

target_review1
P(positive|target_review1): 0.0000
P(negative|target_review1): 0.0000
P(neutral|target_review1): 0.0000

target_review2
P(positive|target_review2): 0.0000
P(negative|target_review2): 1.0000
P(neutral|target_review2): 0.0000


A2-1.   
Target review1의 분류 결과: 모두 아님
Target review2의 분류 결과: 부정적임

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 특정 감정 클래스에서 특정 단어가 등장하지 않을 경우, 그 감정의 가능도가 0이 되어버립니다. 이는 나이브 베이지안의 기본 가정인 조건부 독립성을 깨트리게 되어, 해당 클래스를 전혀 예측할 수 없게 됩니다.
예를 들어, 만약 "awesome"이라는 단어가 'positive' 감정에 대해 학습 데이터에 등장하지 않는다면, 이 단어가 포함된 리뷰에 대해 'positive' 클래스를 전혀 예측할 수 없게 됩니다.

라플라스 스무딩은 제로 확률 문제를 해결하기 위해 사용하는 기법으로, 다음과 같은 원칙에 기반합니다
각 단어의 출현 빈도를 계산할 때, 모든 단어의 빈도에 대해 일정한 작은 값을 추가하여 0이 되는 경우를 방지합니다.